In [1]:
import os
os.chdir('../../..')

In [2]:
import numpy as np
from scipy.integrate import odeint
from sklearn.linear_model import Lasso
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interact
import ipywidgets as widgets

## Observations

In [8]:
# Constants
A = 5
g = 9.81
C = np.sqrt(2*g)/A
q1 = 1
q3 = 1
kv1 = 1
kv2 = 1

In [9]:
# define time steps
t = np.linspace(0, 10, 50)

In [10]:
def system_dynamics_function(x, t, q1, q3, kv1, kv2):
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    dh1_dt =C * q1 - kv1 * C * np.sign(x1 - x2) * np.sqrt(np.abs(x1 - x2))
    dh2_dt = kv1 * C * np.sign(x1 - x2) * np.sqrt(np.abs(x1 - x2)) - kv2 * C * np.sign(x2 - x3) * np.sqrt(np.abs(x2 - x3))
    dh3_dt = C * q3 + kv2 * C* np.sign(x2 - x3) * np.sqrt(np.abs(x2 - x3))
    return dh1_dt, dh2_dt, dh3_dt

In [11]:
q1 = [1, 2]
q3 = [1, 1]
kv1 = [1, 1]
kv2 = [1, 1]

In [12]:
# initial condition
x0 = (10, 100, 33)

In [13]:
# solve ode
y0 = odeint(system_dynamics_function, x0, t, (q1[0], q3[0], kv1[0], kv1[0]))
y1 = odeint(system_dynamics_function, x0, t, (q1[1], q3[1], kv1[1], kv1[1]))

In [15]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                   subplot_titles=("Time series a", "Time series b"))
# signal 1
DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']
names0 = [r'$h_{1,a}(t)$', r'$h_{2,a}(t)$', r'$h_{3,a}(t)$']
names1 = [r'$h_{1,b}(t)$', r'$h_{2,b}(t)$', r'$h_{3,b}(t)$']
# names0 = [r'a', 'c', 'b']
# names1 = [r'a', 'c', 'b']
for color, sig, name in zip(DEFAULT_PLOTLY_COLORS[0:3], [y0[:, 0], y0[:, 1], y0[:, 2]], names0):
    fig.add_trace(go.Scatter(x=t, y=sig, name=name,
                  mode="lines", opacity=1, 
#                              marker_color=color
                            ),
        row=1, col=1)
for color, sig, name in zip(DEFAULT_PLOTLY_COLORS[0:3], [y1[:, 0], y1[:, 1], y1[:, 2]],  names1):
    fig.add_trace(go.Scatter(x=t, y=sig, name=name,
                  mode="lines", opacity=1
#                              , marker_color=color
                            ),
        row=2, col=1)
fig.update_xaxes(title_text=r'Time',row=2, col=1)
fig.update_yaxes(title_text='Fill level')


fig.show()

## Questions one might ask
- Qestion 1-2: How long does it take to fill up all tanks if only v3 is closed an all only q1/q2 is on and tanks are empty before?
- Answer 1: 
  - given fill rate m/s 
  - strecke in m die aufgefüllt werden muss z.B. 3*100 einheiten
  - --> strecke durch rate gleich zeit
- Question 2-4 How long does it take until the tanks run empty if tank1 is full and all valves are open / tank3 is full and v1,v2 are closed?
- Answer 2:
  - tank1:
     - given stand wasser t1 z.b. 100 einheiten
     - erstmal einfach: 100/kv12 + 10/kv23
  - tank2:
     - 100/kv23